<a href="https://colab.research.google.com/github/elsa9421/Optimal-separating-Hyperplanes/blob/master/Hard_Margin_SVM_Demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimal Separating Hyperplanes - Demo1
This Notebook demonstrates Hard Margin SVM. 
For given linearly separable datapoints, the default decision boundary plotted is the hard margin SVM , along with it's respective support vectors and margins. 
<br> On modifying the values of bias `b` or `w_angle` (i.e `atan(w[1]/w[0]`  the change is reflected by plotting the new separting hyperplane `w.T*X+b=0` and it's support vectors and margins.


In [1]:

# Import

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs
from ipywidgets import interact, interactive_output, fixed, interact_manual,interactive
import ipywidgets as widgets
from math import sin,cos,pi,atan,degrees,tan,isclose,radians



# we create 40 separable points

X, y = make_blobs(n_samples=60, centers=[(3, 0), (-3,1)], random_state=6, cluster_std=0.65)
y[y==1]=-1
y[y==0]=1

clf = svm.SVC(kernel='linear',C=1)
y_pred=clf.fit(X, y)


optimal_w=(np.sum(y_pred.coef_**2))**0.5
optimal_theta=atan(y_pred.coef_[0][1]/y_pred.coef_[0][0])

if optimal_theta<0:
   optimal_theta=(2*pi)+optimal_theta
optimal_theta=degrees(optimal_theta)



In [2]:
def valid_hyperplane(X,y,w,b):
  '''
  Checking if it is a valid hyperplane that separates the two classes
  i.e check yi(wT xi + b) > 0 ∀i = 1, . . . , n.

  Input:
  -X: data points, shape(N,2)
  -y: class labels= {-1,1}, shape(N,)
  -w: weights, shape(2,)
  -b: bias 
  
  Output:
  - (bool) True,if valid separting hyperplane exists, otherwise return False
  '''

  output=np.zeros((len(y),))
  wT_x=np.dot(w.T,X.T).T +b       # (N,1)
  output=np.multiply(y,wT_x)      #(N,1)

  
  if all(i >0 for i in output) or all(i <0 for i in output):
    return True
  else:
    return False


def find_vectors2(X,y,w,b):
  
  '''
  Find and return closest data points to decision boundary for input X,y,w,b
  decision_boundary = -w0/w1 * x_vec - bias/w1

  |r| =|wT z + b|/||w||

  Input:
  -X: data points, shape(N,2)
  -y: class labels= {-1,1}, shape(N,)
  -w: weights, shape(2,)
  -b: bias 

  Output:
  -sv: support vectors
  -closest: support vector with min distance to separating hyperplane

  '''


  dist=np.zeros((len(X),))
  dist=np.divide(abs(np.dot(w.T,X.T).T+b),np.linalg.norm(w))
  sv=[]
  dist_sort=np.argsort(dist)
  sv.append(X[dist_sort[0]])


  k=1
  while(isclose(dist[dist_sort[k-1]], dist[dist_sort[k]], abs_tol=10**-2) ):
    sv.append(X[dist_sort[k]])
    k=k+1
    if k>=len(dist):
      break

  sv=np.array(sv)
  closest=sv[0]
  

  
  return closest,sv


def plot_svc(w_theta,bias=y_pred.intercept_,X=X,y=y,w_mag=optimal_w):
  '''
  Function to plot the decision boundary, margins and support vectors for changing
  input values of w_theta and bias

  '''

    w_theta_rad=radians(w_theta)
    w0=w_mag*cos(w_theta_rad)
    w1=w_mag*sin(w_theta_rad)
    

    clf = svm.SVC(kernel='linear')
    y_pred=clf.fit(X, y)
    
    
    w=np.array([w0,w1])

    ####

    valid=valid_hyperplane(X,y,w,bias)
    if (valid):
      plt.figure(figsize=(7,7))
      plt.scatter(X[:, 0], X[:, 1], c=y, s=30)
      x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
      y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    
      plt.xlim(x_min,x_max)
      plt.ylim(y_min,y_max)


      x_vec = np.linspace(x_min, x_max, 200)

      #print("Valid separating hyperplane")


      closest,svs=find_vectors2(X,y,w,bias)


      # At the decision boundary, w0*x0 + w1*x1 + b = 0
      # => x1 = -w0/w1 * x0 - b/w1
      slope=-w[0]/w[1]
      decision_boundary = slope * x_vec - bias/w[1]
      # To plot margin through support vector
      intercept_up=closest[1]-(slope*closest[0])
      intercept_down=-closest[0]+(slope*closest[1])
      margin_up=slope * x_vec +intercept_up   # line through support vector
      margin_down=decision_boundary-intercept_up-(bias/w[1]) 
      
      plt.scatter(svs[:, 0], svs[:, 1], s=180, facecolors='none', edgecolors='k')
      plt.plot(x_vec, decision_boundary, "k-", linewidth=2)
      plt.plot(x_vec, margin_up, "k--", linewidth=2)
      plt.plot(x_vec, margin_down, "k--", linewidth=2)
    
    else:
      print("Not a valid separating hyperplane for given value of theta")



    

In [4]:
w_theta_slider=widgets.FloatSlider(value=optimal_theta,
                                 min=298,
                                 max=400,
                                 step=1,
                                 description='w_theta',
                                 continuous_update=False)

w_theta_text=widgets.FloatText(value=optimal_theta,
                                 min=298,
                                 max=400,
                                 step=1,
                                 description='w_theta',
                                 continuous_update=False)

widgets.link((w_theta_slider, 'value'), (w_theta_text, 'value'))
w_theta_widget=widgets.HBox([w_theta_slider,w_theta_text])

bias_slider=widgets.FloatSlider(value=y_pred.intercept_,
                                 min=-1,
                                 max=1,
                                 step=.1,
                                 description='bias',
                                 continuous_update=False)

bias_text=widgets.FloatText(value=y_pred.intercept_,
                                 min=-1,
                                 max=1,
                                 step=0.1,
                                 description='bias',
                                 continuous_update=False)

widgets.link((bias_slider, 'value'), (bias_text, 'value'))
bias_widget=widgets.HBox([bias_slider,bias_text])

out=interactive_output(plot_svc,{"w_theta":w_theta_text,"bias":bias_text})
display(w_theta_widget,bias_widget,out)



Output()